# Практическое задание 3

## Замечания
* Можно (и нужно!) применять для реализации только библиотеку **Numpy**
* Ничего, крому Numpy, нельзя использовать для реализации 
* **Keras** используется только для тестирования Вашей реализации
* Если какой-то из классов не проходит приведенные тесты, то соответствующее задание не оценивается
* Возможно использование дополнительных (приватных) тестов
 

In [1]:
# Вам понадобится для реализации
import numpy as np
# Нужно для тестирования
from tensorflow import keras
import keras.layers as layers

* Вспомогательные функции для тестирования

In [2]:
def compare_tensors(x, y, tol=0.001, test_name='Test'):
  assert (x.shape == y.shape), test_name + ' different shapes'
  diff = np.sum((y - x)**2)
  assert (diff < tol), test_name + ' Failed!'
  print (test_name + ' Passed!')
  return

In [3]:
def compare_tensors_array(x, y, tol=0.001, test_name='Test'):
  assert (len(x) == len(y)), test_name + ' different lengths'
  for i in range(len(x)):
    t = test_name + ' subtest ' + str(i)
    compare_tensors(x[i], y[i], tol=tol, test_name=t)
  print (test_name + ' Passed!')
  return

* Шаблон класса любой операции (слоя), которую Вам необходимо будет реализовать

In [4]:
class Layer(object):
    def __init__(self):
        self.name = 'Layer'       
    def forward(self, input_data):
        pass



---



* (1 балл) Реализация "спрямляющего" слоя Flatten

In [5]:
class FlattenLayer(Layer):
    def __init__(self):
      self.name = 'Flatten'
    
    def forward(self, input_data):
      # На входе - четырехмерный тензор вида [batch, input_channels, height, width]
      # Преобразуем в двухмерный тензор: при этом по первой размерности НЕ преобразуем
      # Выкладываем данные: сначала по последней размерности, затем по предпоследней и т.д.
      # Нужно заполнить Nu mpy-тензор out  
      in_shape = np.array(input_data).shape
      out = []
        
      for batch in range(in_shape[0]):
          raws = []
          new_raw = []
          
          for raw_idx in range(in_shape[2]):
              raws = [input_data[batch][channel][raw_idx] for channel in range(in_shape[1])]
              raws = np.stack(raws, axis=1)
              raws = np.concatenate(raws)
              new_raw.append(raws)
                
          new_raw = np.array(new_raw).reshape(-1)
          out.append(new_raw)
            
      out = np.array(out) 
      # out = np.empty([])
      return out

* Функция предварительного тестирования слоя **Flatten**
* Функции с названием "**test_**" не менять
* Вы можете самостоятельно поиграться с параметрами типа B/C/H/W etc

In [6]:
def test_FlattenLayer():
  B = 1
  C = 1
  H = 3
  W = 3
  x = np.random.randn(B, C, H, W)
  y = layers.Flatten(data_format='channels_first')
  y_keras = y(x).numpy()
  y_out = FlattenLayer().forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test Flatten 1')
  B = 1
  C = 2
  H = 3
  W = 3
  x = np.random.randn(B, C, H, W)
  y = layers.Flatten(data_format='channels_first')
  y_keras = y(x).numpy()
  y_out = FlattenLayer().forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test Flatten 2')
  return

* Запуск теста слоя Flatten
* Нужно, чтобы все тесты были '*Passed!*'

In [7]:
test_FlattenLayer()

Test Flatten 1 Passed!
Test Flatten 2 Passed!




---



* (1 балл) Реализация слоя субдискретизации **Global Average Pooling**

In [1]:
class GAP2DLayer(Layer):
    def __init__(self):
      self.name = 'GAP2D'
    
    def forward(self, input_data):
      # На входе - четырехмерный тензор вида [batch, input_channels, height, width]
      # Сворачиваем по двум последним размерностям (то есть на выходе - минус две размерности)
      # Нужно заполнить Numpy-тензор out 
        shape = input_data.shape
        out = []
        for batch in range(shape[0]):
            channel_arr = []

            for channel in range(shape[1]):
                w_sum = np.array([np.sum(arr) for arr in input_data[batch][channel]])
                w_divider = shape[3]
                w_result = w_sum / w_divider

                val_divider = shape[2]
                final_val = np.sum(w_result) / val_divider
                channel_arr.append(final_val)

            out.append(channel_arr)
 
        out = np.array(out)
        # out = np.empty([])
        return out

NameError: name 'Layer' is not defined

In [9]:
def test_GAP2DLayer():
  B = 1
  C = 1
  H = 3
  W = 3
  x = np.random.randn(B, C, H, W)
  y = layers.GlobalAveragePooling2D(data_format='channels_first')
  y_keras = y(x).numpy()
  y_out = GAP2DLayer().forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test GAP2D 1')
  B = 1
  C = 2
  H = 3
  W = 3
  x = np.random.randn(B, C, H, W)
  y = layers.GlobalAveragePooling2D(data_format='channels_first')
  y_keras = y(x).numpy()
  y_out = GAP2DLayer().forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test GAP2D 2')
  return

In [10]:
test_GAP2DLayer()

Test GAP2D 1 Passed!
Test GAP2D 2 Passed!




---



* (2 балла) Реализация слоя субдискретизации **MaxPooling**

In [10]:
class MaxPool2DLayer(Layer):
    def __init__(self, pool_size=2, stride=2):
      self.name = 'MaxPool2D'
      self.pool_size = pool_size
      self.stride = stride
    def forward(self, input_data):
      # На входе - четырехмерный тензор вида [batch, input_channels, height, width]
      # Нужно заполнить Numpy-тензор out 
      # out = np.empty([])
      shape = input_data.shape
      out = []
      for batch in range(shape[0]):
          batch_arr = []
          for channel in range(shape[1]):
              channel_arr = []
              vals = []
              for raw in range(0, shape[2], self.stride):
                  if raw + self.pool_size > shape[2]:
                      break
                  pooled_data = [np.max(input_data[batch][channel][raw:raw+self.pool_size, col:col+self.pool_size])
                                        for col in range(0, shape[3], self.stride)
                                        if col + self.pool_size <= shape[3]]
                  channel_arr.append(pooled_data)
              channel_arr = np.array(channel_arr).reshape(-1)
              channel_arr = np.expand_dims(channel_arr, axis=1)
              batch_arr.append(channel_arr)
          out.append(batch_arr)
      out = np.array(out)
      return out

In [13]:
def test_MaxPool2DLayer():
  B = 1
  C = 1
  H = 4
  W = 4
  pool_size = 2
  stride = 2
  x = np.random.randn(B, C, H, W)
#   y = layers.MaxPooling2D(pool_size=pool_size, strides=stride, padding="valid", data_format='channels_first')
#   y_keras = y(x).numpy()
  y_out = MaxPool2DLayer(pool_size=pool_size, stride=stride).forward(x)
#   compare_tensors(y_keras, y_out, tol=0.001, test_name='Test MaxPool2D 1')
  B = 2
  C = 2
  H = 3
  W = 3
  pool_size = 2
  stride = 1  
  x = np.random.randn(B, C, H, W)
#   y = layers.MaxPooling2D(pool_size=pool_size, strides=stride, padding="valid", data_format='channels_first')
#   y_keras = y(x).numpy()
  y_out = MaxPool2DLayer(pool_size=pool_size, stride=stride).forward(x)
#   compare_tensors(y_keras, y_out, tol=0.001, test_name='Test MaxPool2D 2')
  return

In [14]:
test_MaxPool2DLayer()

[[[[ 1.19373037 -0.47563104 -1.80720975]
   [-0.97288308 -0.36535866  1.0487244 ]
   [-0.89148072 -1.88005553  2.04017067]]

  [[-2.19326417  1.00409025  0.6323169 ]
   [-0.76396745  0.67916278  0.4903245 ]
   [-0.87940529 -0.13046821 -0.43074683]]]


 [[[ 1.5291461   0.85049268  1.33586984]
   [-1.36521824  0.24638333 -1.9505574 ]
   [ 0.57913739  0.66941551 -0.08326311]]

  [[ 0.41793839  0.16968227 -0.94651394]
   [-0.04385755 -1.69168188 -0.88913239]
   [ 0.41828686 -0.13152112  2.64323824]]]]
[[[[ 1.19373037]
   [ 1.0487244 ]
   [-0.36535866]
   [ 2.04017067]]

  [[ 1.00409025]
   [ 1.00409025]
   [ 0.67916278]
   [ 0.67916278]]]


 [[[ 1.5291461 ]
   [ 1.33586984]
   [ 0.66941551]
   [ 0.66941551]]

  [[ 0.41793839]
   [ 0.16968227]
   [ 0.41828686]
   [ 2.64323824]]]]




---



* (3 балла) Реализация слоя **активации** (поддерживаются **relu**, **sigmoid**, **softmax**)

In [14]:
import math
class ActivationLayer(Layer):
    def __init__(self, activation='relu'):
      # Активация (поддерживаем 'relu', 'sigmoid', 'softmax')
      self.name = 'Activation'
      self.activation = activation
    
    def forward(self, input_data):   
      # На входе:
      # четырехмерный тензор вида [batch, input_channels, height, width] для 'relu', 'sigmoid'
      # или двухмерный тензор вида [batch, logits]
      # SoftMax применяется по последней размерности
      # Нужно заполнить Numpy-тензор out 
      # out = np.empty([])
          
      shape = input_data.shape
      out = []
        
      if self.activation != 'softmax':
          func_dict = {
              'relu': lambda x: x if x>0 else 0,
              'sigmoid': lambda x: 1/(1+math.e**(-x)),
          }

          for batch in range(shape[0]):
              batch_arr = []
              for channel in range(shape[1]):
                  channel_arr = []
                  divider = 0
                  for raw in range(shape[2]):
                      activated = [func_dict[self.activation](input_data[batch, channel, raw, col]) for col in range(shape[3])]
                      channel_arr.append(activated)
                  batch_arr.append(channel_arr)  
              out.append(batch_arr)
      else:
        for batch in range(shape[0]):
            exponents = np.exp(input_data[batch])
            divider = sum(exponents)
            out.append(exponents/divider)
      out = np.array(out)
      return out

In [15]:
def test_ActivationLayer():
  B = 1
  C = 1
  H = 4
  W = 4
  activation = 'relu'
  x = np.random.randn(B, C, H, W)
  y = layers.Activation(activation)
  y_keras = y(x).numpy()
  y_out = ActivationLayer(activation=activation).forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test Activation 1')
  B = 2
  C = 2
  H = 3
  W = 3
  activation = 'sigmoid'  
  x = np.random.randn(B, C, H, W)
  y = layers.Activation(activation)
  y_keras = y(x).numpy()
  y_out = ActivationLayer(activation=activation).forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test Activation 2')
  B = 3
  C = 10
  activation = 'softmax'
  x = np.random.randn(B, C)
  y = layers.Activation(activation)
  y_keras = y(x).numpy()
  y_out = ActivationLayer(activation=activation).forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test Activation 3')  
  return

In [16]:
test_ActivationLayer()

Test Activation 1 Passed!
Test Activation 2 Passed!
Test Activation 3 Passed!




---



* (3 балла) Реализация слоя пакетной нормализации **BatchNorm** (как для режима train, так и для режима test)

In [17]:
# Hint
# Train mode:
# out = (batch - mean(batch)) / sqrt(var(batch) + epsilon) * gamma + beta
# moving_mean = moving_mean * momentum + mean(batch) * (1 - momentum)
# moving_var = moving_var * momentum + var(batch) * (1 - momentum)
# Test mode:
# (batch - moving_mean) / sqrt(moving_var + epsilon) * gamma + beta

class BatchNormLayer(Layer):
    def __init__(self, momentum=0.99, epsilon=0.001, beta_init=None, gamma_init=None,
                 moving_mean_init=None, moving_var_init=None,
                 mode='train', input_channels=2):
      # mode: 'train', 'test'
      # Параметры gamma, beta, mean, var - все имеют размерность по количеству карт input_channels   
      self.name = 'BatchNorm'
      self.momentum = momentum
      self.epsilon = epsilon
      self.beta = beta_init
      self.gamma = gamma_init
      self.moving_mean = moving_mean_init
      self.moving_var = moving_var_init
      self.mode = mode
      self.input_channels = input_channels
    
    def forward(self, input_data):   
      # На входе - четырехмерный тензор вида [batch, input_channels, height, width]
      # 1) Нужно заполнить Numpy-тензор out (той же размерности, что и вход)
      # 2) Нужно обновить moving_mean и moving_var в режиме 'train'
      # out = np.empty([])

      shape = input_data.shape
      out = []
      mean = []
      var = []
    
      if self.mode == 'train':
          mean = [np.mean(input_data[:, i]) for i in range(shape[1])]
          var = [np.var(input_data[:, i]) for i in range(shape[1])]
          self.moving_mean = np.array([self.moving_mean[i] * self.momentum + mean[i] * (1 - self.momentum) for i in range(self.input_channels)])
          self.moving_var = np.array([self.moving_var[i] * self.momentum + var[i] * (1 - self.momentum) for i in range(self.input_channels)])
            
      elif self.mode == 'test':
          mean = self.moving_mean
          var = self.moving_var
            
      for batch in range(shape[0]):
          batch_arr = []
          for channel in range(shape[1]):
              result = self.gamma[channel] * (input_data[batch, channel] - mean[channel]) / np.sqrt(var[channel] + self.epsilon) + self.beta[channel]
              batch_arr.append(result)
          out.append(batch_arr)
          
      out = np.array(out)
      return out

In [18]:
def test_BatchNormLayer():
  B = 2
  C = 2
  H = 4
  W = 4
  beta_init = 0 * np.ones(C)
  gamma_init = 1 * np.ones(C)
  moving_mean_init = 0 * np.ones(C)
  moving_var_init= 1 * np.ones(C)
  momentum = 0.99
  epsilon = 0.001
  mode = 'train'
  x = np.random.randn(B, C, H, W)
  y = layers.BatchNormalization(axis=1, momentum=momentum, epsilon=epsilon, trainable=True)
  y_keras = y(x, training=True).numpy()
  y.set_weights([gamma_init, beta_init, moving_mean_init, moving_var_init])
  y_keras = y(x, training=True).numpy()
  y_out_layer = BatchNormLayer(momentum=momentum, epsilon=epsilon, beta_init=beta_init, gamma_init=gamma_init,
                 moving_mean_init=moving_mean_init, moving_var_init=moving_var_init,
                 mode=mode, input_channels=C)
  y_out = y_out_layer.forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test BatchNorm 1')
  compare_tensors_array(y.get_weights(), 
                        [y_out_layer.gamma, y_out_layer.beta, y_out_layer.moving_mean, y_out_layer.moving_var],
                        tol=0.00001, test_name='Test BatchNorm 1.1')

  B = 2 
  C = 2 
  H = 4 
  W = 4 
  beta_init = 1 * np.ones(C)
  gamma_init = 0 * np.ones(C)
  moving_mean = 0 * np.ones(C)
  moving_var = 1 * np.ones(C)
  momentum = 0.99
  epsilon = 0.001
  mode = 'test'
  x = np.random.randn(B, C, H, W)
  y = layers.BatchNormalization(axis=1, momentum=momentum, epsilon=epsilon, trainable=False)
  y_keras = y(x, training=False).numpy()
  y.set_weights([gamma_init, beta_init, moving_mean, moving_var])
  y_keras = y(x, training=False).numpy()
  y_out_layer = BatchNormLayer(momentum=momentum, epsilon=epsilon, beta_init=beta_init, gamma_init=gamma_init,
                 moving_mean_init=moving_mean, moving_var_init=moving_var,
                 mode=mode, input_channels=C)
  y_out = y_out_layer.forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test BatchNorm 2')  
  compare_tensors_array(y.get_weights(), 
                        [y_out_layer.gamma, y_out_layer.beta, y_out_layer.moving_mean, y_out_layer.moving_var],
                        tol=0.00001, test_name='Test BatchNorm 2.1')
  return

In [19]:
test_BatchNormLayer()

Test BatchNorm 1 Passed!
Test BatchNorm 1.1 subtest 0 Passed!
Test BatchNorm 1.1 subtest 1 Passed!
Test BatchNorm 1.1 subtest 2 Passed!
Test BatchNorm 1.1 subtest 3 Passed!
Test BatchNorm 1.1 Passed!
Test BatchNorm 2 Passed!
Test BatchNorm 2.1 subtest 0 Passed!
Test BatchNorm 2.1 subtest 1 Passed!
Test BatchNorm 2.1 subtest 2 Passed!
Test BatchNorm 2.1 subtest 3 Passed!
Test BatchNorm 2.1 Passed!




---



* (1 балл) Реализация **полносвязного** слоя

In [17]:
class DenseLayer(Layer):
    def __init__(self, input_dim, output_dim, W_init=None, b_init=None):
      self.name = 'Dense'
      self.input_dim = input_dim
      self.output_dim = output_dim
      self.W = W_init
      self.b = b_init
    
    def forward(self, input_data):
      # На входе - двухмерный тензор вида [batch, input_channels]
      # Работаем по второй размерности, по первой размерности НЕ преобразуем
      # Вначале нужно проверить на согласование размерностей входных данных и ядра!
      # Нужно заполнить Numpy-тензор out 
      out = np.empty([]) #
      assert input_data.shape[-1] == self.W.shape[0] == self.input_dim
      out = input_data @ self.W + self.b
      return out

In [18]:
def test_DenseLayer():
  B = 1
  C_IN = 10
  C_OUT = 5
  x = np.random.randn(B, C_IN)
  W_init = np.random.randn(C_IN, C_OUT)
  b_init = np.random.randn(C_OUT)
  y = layers.Dense(C_OUT, use_bias=True)
  y_keras = y(x).numpy()
  y.set_weights([W_init, b_init])
  y_keras = y(x).numpy()
  y_out = DenseLayer(C_IN, C_OUT, W_init=W_init, b_init=b_init).forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test Dense 1')
  B = 2
  C_IN = 5
  C_OUT = 10
  x = np.random.randn(B, C_IN)
  W_init = np.random.randn(C_IN, C_OUT)
  b_init = np.random.randn(C_OUT)
  y = layers.Dense(C_OUT, use_bias=True, input_shape=(C_IN,))
  y_keras = y(x).numpy()
  y.set_weights([W_init, b_init])
  y_keras = y(x).numpy()
  print(y_keras)
  y_out = DenseLayer(C_IN, C_OUT, W_init=W_init, b_init=b_init).forward(x)
  print(y_out)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test Dense 2')
  return

In [ ]:
test_DenseLayer()



---



* (2 балла) Реализация **сверточного** слоя

In [37]:
class Conv2DLayer(Layer):
    def __init__(self, kernel_size=3, input_channels=2, output_channels=3, 
                 padding='same', stride=1, K_init=None, b_init=None):
      # padding: 'same' или 'valid'
      # Работаем с квадратными ядрами, поэтому kernel_size - одно число
      # Работаем с единообразным сдвигом, поэтому stride - одно число
      # Фильтр размерности [kernel_size, kernel_size, input_channels, output_channels]
      self.name = 'Conv2D'
      self.kernel_size = kernel_size
      self.input_channels = input_channels
      self.output_channels = output_channels
      self.kernel = K_init
      self.bias = b_init
      self.padding = padding
      self.stride = stride
    def forward(self, input_data):
      # На входе - четырехмерный тензор вида [batch, input_channels, height, width]
      # Вначале нужно проверить на согласование размерностей входных данных и ядра!
      # Нужно заполнить Numpy-тензор out 
#       assert self.kernel_size[-1] == self.output_channels, self.kernel_size[-2] == self.input_channels
#       assert input_data[1] == self.input_channels
      
      return data

In [48]:
def test_Conv2DLayer():
  B = 1
  C_IN = 1
  C_OUT = 1
  H = 10
  W = 10
  K = 3
  S = 1
  padding = 'same'
  x = np.random.randn(B, C_IN, H, W)
  K_init = np.random.randn(K, K, C_IN, C_OUT)
  b_init = np.random.randn(C_OUT)
  y = layers.Conv2D(C_OUT, K, strides=S, padding=padding, data_format='channels_first',
    dilation_rate=1, groups=1, activation=None, use_bias=True)
  y_keras = y(x).numpy()
  y.set_weights([K_init, b_init])
  y_keras = y(x).numpy()
  print(x)
  print(K_init)
  print(b_init)
  print(y_keras)

#   y_out = Conv2DLayer(kernel_size=K, input_channels=C_IN, output_channels=C_OUT, 
#                  padding=padding, stride=S, K_init=K_init, b_init=b_init).forward(x)
#   print(y_out)
#   compare_tensors(y_keras, y_out, tol=0.001, test_name='Test Conv2D 1')

#   B = 2
#   C_IN = 3
#   C_OUT = 5
#   H = 9
#   W = 9
#   K = 3
#   S = 2
#   padding = 'valid'
#   x = np.random.randn(B, C_IN, H, W)
#   K_init = np.random.randn(K, K, C_IN, C_OUT)
#   b_init = np.random.randn(C_OUT)
#   y = layers.Conv2D(C_OUT, K, strides=S, padding=padding, data_format='channels_first',
#     dilation_rate=1, groups=1, activation=None, use_bias=True, input_shape=(C_IN, H, W))
#   y_keras = y(x).numpy()
#   y.set_weights([K_init, b_init])
#   y_keras = y(x).numpy()
#   y_out = Conv2DLayer(kernel_size=K, input_channels=C_IN, output_channels=C_OUT, 
#                  padding=padding, stride=S, K_init=K_init, b_init=b_init).forward(x)
#   print(x)
#   print(K_init)
#   print (b_init)
#   print(y_keras)
#   print(y_out)
#   compare_tensors(y_keras, y_out, tol=0.001, test_name='Test Conv2D 1')
  return

In [49]:
test_Conv2DLayer()

[[[[ 0.33810245  0.50561549 -2.08248016  0.23325591  0.53277484
     1.97011148 -0.37944922 -1.34314863  2.46978494  1.11003021]
   [ 0.27297025 -0.32563047 -0.07173462  0.46644386 -0.56924185
    -0.99514498 -1.31138017 -0.65533014  0.80680934 -2.07072258]
   [ 0.11522915  0.49470898 -0.19826275  2.38524529  0.69625707
     0.59525819  0.01456653 -0.76295249  0.22539998  0.9780864 ]
   [-1.19762031 -0.12016672 -0.39350596 -0.65021846 -0.04573665
    -0.50577363  1.89160395 -2.72709064  1.01974559  0.94212133]
   [ 0.07368161  0.14014638  0.33364052  0.92671546  0.6037298
     1.29983793  0.48106357  2.32106652  1.69256248 -1.93717853]
   [ 0.67978298  0.25327346 -2.66800411 -0.94412671 -0.26273395
    -0.20955409 -0.75756119 -0.850852    1.33903918 -2.36113146]
   [-0.14234271  0.63042205  0.70899265 -0.77700189 -0.5914539
     1.00591701  1.95847309 -0.70882734 -0.45067135  0.94307293]
   [ 1.05134908 -1.07586943  0.74146745 -0.90691587 -2.87408321
    -2.37739167 -2.06621606 -2.5341



---



* (2 балла) Реализация **транспонированного сверточного** слоя

In [ ]:
class Conv2DTrLayer(Layer):
    def __init__(self, kernel_size=3, input_channels=2, output_channels=3, 
                 padding=0, stride=1, K_init=None, b_init=None):      
      # padding: число (сколько отрезать от модифицированной входной карты)
      # Работаем с квадратными ядрами, поэтому kernel_size - одно число
      # stride - одно число (коэффициент расширения)
      # Фильтр размерности [kernel_size, kernel_size, input_channels, output_channels]
      self.name = 'Conv2DTr'
      self.kernel_size = kernel_size
      self.input_channels = input_channels
      self.output_channels = output_channels
      self.kernel = K_init
      self.bias = b_init
      self.padding = padding
      self.stride = stride
    def forward(self, input_data):
      # На входе - четырехмерный тензор вида [batch, input_channels, height, width]
      # Вначале нужно проверить на согласование размерностей входных данных и ядра!
      # Нужно заполнить Numpy-тензор out 
      out = np.empty([])
      return out

In [ ]:
def adjust_kernel(K):
  K_new = K.copy()[::-1, ::-1, :, :]
  K_new = np.transpose(K_new, (0, 1, 3, 2))
  return K_new

def test_Conv2DTrLayer():
  B = 1
  C_IN = 1
  C_OUT = 1
  H = 3
  W = 3
  K = 3
  S = 2
  padding = 0
  x = np.random.randn(B, C_IN, H, W)
  K_init = np.random.randn(K, K, C_IN, C_OUT)
  b_init = np.random.randn(C_OUT)
  y = layers.Conv2DTranspose(C_OUT, K, strides=S, padding="valid", output_padding=None, 
                    data_format='channels_first', dilation_rate=1, groups=1, 
                    activation=None, use_bias=True)
  y_keras = y(x).numpy()
  y.set_weights([adjust_kernel(K_init), b_init])
  y_keras = y(x).numpy()
  y_out = Conv2DTrLayer(kernel_size=K, input_channels=C_IN, output_channels=C_OUT, 
                 padding=padding, stride=S, K_init=K_init, b_init=b_init).forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test Conv2DTr 1')
  B = 4
  C_IN = 2
  C_OUT = 3
  H = 3
  W = 3
  K = 3
  S = 2
  padding = 0
  x = np.random.randn(B, C_IN, H, W)
  K_init = np.random.randn(K, K, C_IN, C_OUT)
  b_init = np.random.randn(C_OUT)
  y = layers.Conv2DTranspose(C_OUT, K, strides=S, padding="valid", output_padding=None, 
                    data_format='channels_first', dilation_rate=1, groups=1, 
                    activation=None, use_bias=True)
  y_keras = y(x).numpy()
  y.set_weights([adjust_kernel(K_init), b_init])
  y_keras = y(x).numpy()
  y_out = Conv2DTrLayer(kernel_size=K, input_channels=C_IN, output_channels=C_OUT, 
                 padding=padding, stride=S, K_init=K_init, b_init=b_init).forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test Conv2DTr 2')
  return

In [ ]:
test_Conv2DTrLayer()